# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv('cities_df.csv')
cities = cities.drop(['Unnamed: 0'],axis=1)
cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,100,NZ,1580548551,63,-46.1927,168.8643,71.01,11.99
1,Bredasdorp,90,ZA,1580547935,73,-34.5322,20.0403,75.99,5.99
2,Sorland,75,NO,1580548435,80,67.6670,12.6934,39.20,24.16
3,Tura,0,IN,1580548155,37,25.5198,90.2201,75.24,6.80
4,Hobart,75,AU,1580548144,93,-42.8794,147.3294,64.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(g_key)

fig = gmaps.figure(map_type='HYBRID')

locations = cities[['Lat','Lng','Humidity']]
heatmap_layer = gmaps.heatmap_layer(locations[['Lat','Lng']], weights = locations['Humidity'],max_intensity=30, point_radius=3.0)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
cities.columns


Index(['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng',
       'Max Temp', 'Wind Speed'],
      dtype='object')

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
fit_citiesdf = cities.loc[(cities['Cloudiness'] == 0) & (cities['Humidity'] < 50) &
           (cities['Max Temp']<75) & (cities['Wind Speed'] < 10), :]

fit_citiesdf = fit_citiesdf.reset_index().drop(['index'],axis=1)
fit_citiesdf

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Guerrero Negro,0,MX,1580548489,35,27.9769,-114.0611,61.86,9.26
1,Umm Lajj,0,SA,1580548143,15,25.0213,37.2685,73.35,2.82
2,Sardarshahr,0,IN,1580548156,27,28.4333,74.4833,68.00,3.94
3,Sur,0,OM,1580548145,41,22.5667,59.5289,73.74,5.91
4,Chenghai,0,CN,1580548153,39,23.4632,116.7680,64.40,8.95
5,Khalabat,0,PK,1580548154,39,34.0600,72.8894,62.60,4.70
6,Chhatapur,0,IN,1580548159,30,26.2167,87.0167,73.31,6.64
7,El Reno,0,US,1580548505,44,35.5323,-97.9550,36.00,6.93
8,Ures,0,MX,1580548489,33,29.4333,-110.4000,55.00,3.00
9,Petropavlovsk-Kamchatskiy,0,RU,1580548168,46,53.0452,158.6483,8.60,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = fit_citiesdf
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Guerrero Negro,0,MX,1580548489,35,27.9769,-114.0611,61.86,9.26
1,Umm Lajj,0,SA,1580548143,15,25.0213,37.2685,73.35,2.82
2,Sardarshahr,0,IN,1580548156,27,28.4333,74.4833,68.00,3.94
3,Sur,0,OM,1580548145,41,22.5667,59.5289,73.74,5.91
4,Chenghai,0,CN,1580548153,39,23.4632,116.7680,64.40,8.95
5,Khalabat,0,PK,1580548154,39,34.0600,72.8894,62.60,4.70
6,Chhatapur,0,IN,1580548159,30,26.2167,87.0167,73.31,6.64
7,El Reno,0,US,1580548505,44,35.5323,-97.9550,36.00,6.93
8,Ures,0,MX,1580548489,33,29.4333,-110.4000,55.00,3.00
9,Petropavlovsk-Kamchatskiy,0,RU,1580548168,46,53.0452,158.6483,8.60,4.47


In [38]:
hotel_df['Lat_Lng'] = hotel_df["Lat"].apply(str) + str(", ") + hotel_df["Lng"].apply(str)
coordinates = hotel_df['Lat_Lng'].apply(str)
coordinates

0              27.9769, -114.0611
1                25.0213, 37.2685
2                28.4333, 74.4833
3                22.5667, 59.5289
4     23.4632, 116.76799999999999
5                  34.06, 72.8894
6                26.2167, 87.0167
7                35.5323, -97.955
8                 29.4333, -110.4
9               53.0452, 158.6483
10             29.9833, -109.7833
11               31.3411, 69.4481
12               22.6783, -3.9836
13               32.8663, 59.2211
14               36.2133, 58.7958
15                  25.2333, 77.6
16               32.2501, 54.0166
Name: Lat_Lng, dtype: object

In [39]:


# find the closest lodging of each type to coordinates

hotels = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for coord in coordinates:
    params = {
    "location": str(coord),
    "keyword" : "hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

    response = requests.get(base_url, params=params).json()
    
    try:
        hotelsel = response['results'][0]['name']
   
        print(f"Closest lodging to {coord} is {hotelsel}.")
        
        hotels.append(hotelsel)
        
    except (IndexError):
        hotels.append('NaN')
        print("Missing field/result... skipping.")
          
hotels
    


Closest lodging to 27.9769, -114.0611 is The Halfway Inn.
Closest lodging to 25.0213, 37.2685 is عروس الشاطئ للوحدات السكنية المفروشة.
Closest lodging to 28.4333, 74.4833 is Kuber Palace Hotel.
Closest lodging to 22.5667, 59.5289 is Sur Plaza Hotel.
Closest lodging to 23.4632, 116.76799999999999 is Garden Hotel Chenghai.
Missing field/result... skipping.
Closest lodging to 26.2167, 87.0167 is Namah Shivay Hotel.
Closest lodging to 35.5323, -97.955 is Home2 Suites by Hilton El Reno.
Closest lodging to 29.4333, -110.4 is Hacienda del Labrador.
Closest lodging to 53.0452, 158.6483 is Avacha Hotel.
Closest lodging to 29.9833, -109.7833 is Hotel La Palma.
Closest lodging to 31.3411, 69.4481 is Faiz Hotel and Resturant.
Missing field/result... skipping.
Closest lodging to 32.8663, 59.2211 is Sepehr Hotel.
Closest lodging to 36.2133, 58.7958 is Amiran Hotel and Grand Complex.
Missing field/result... skipping.
Closest lodging to 32.2501, 54.0166 is Traditional Hotel Restaurant Haji Malek.


['The Halfway Inn',
 'عروس الشاطئ للوحدات السكنية المفروشة',
 'Kuber Palace Hotel',
 'Sur Plaza Hotel',
 'Garden Hotel Chenghai',
 'NaN',
 'Namah Shivay Hotel',
 'Home2 Suites by Hilton El Reno',
 'Hacienda del Labrador',
 'Avacha Hotel',
 'Hotel La Palma',
 'Faiz Hotel and Resturant',
 'NaN',
 'Sepehr Hotel',
 'Amiran Hotel and Grand Complex',
 'NaN',
 'Traditional Hotel Restaurant Haji Malek']

In [40]:
hotel_df.insert(9,'Hotel Name',hotels)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Lat_Lng
0,Guerrero Negro,0,MX,1580548489,35,27.9769,-114.0611,61.86,9.26,The Halfway Inn,"27.9769, -114.0611"
1,Umm Lajj,0,SA,1580548143,15,25.0213,37.2685,73.35,2.82,عروس الشاطئ للوحدات السكنية المفروشة,"25.0213, 37.2685"
2,Sardarshahr,0,IN,1580548156,27,28.4333,74.4833,68.00,3.94,Kuber Palace Hotel,"28.4333, 74.4833"
3,Sur,0,OM,1580548145,41,22.5667,59.5289,73.74,5.91,Sur Plaza Hotel,"22.5667, 59.5289"
4,Chenghai,0,CN,1580548153,39,23.4632,116.7680,64.40,8.95,Garden Hotel Chenghai,"23.4632, 116.76799999999999"
5,Khalabat,0,PK,1580548154,39,34.0600,72.8894,62.60,4.70,NaN,"34.06, 72.8894"
6,Chhatapur,0,IN,1580548159,30,26.2167,87.0167,73.31,6.64,Namah Shivay Hotel,"26.2167, 87.0167"
7,El Reno,0,US,1580548505,44,35.5323,-97.9550,36.00,6.93,Home2 Suites by Hilton El Reno,"35.5323, -97.955"
8,Ures,0,MX,1580548489,33,29.4333,-110.4000,55.00,3.00,Hacienda del Labrador,"29.4333, -110.4"
9,Petropavlovsk-Kamchatskiy,0,RU,1580548168,46,53.0452,158.6483,8.60,4.47,Avacha Hotel,"53.0452, 158.6483"


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))